In [3]:
from FUCCIDataset import ReferenceChannelDataset
FUCCI_PATH = '/data/ishang/Fucci-dataset-v3_filtered'
res = 256

In [4]:
dataset = ReferenceChannelDataset(FUCCI_PATH, imsize=res)

Takes 36 seconds to load the dataset for a given process

In [5]:
print(f"total: {len(dataset)}")
print(f"train: {int(len(dataset) * 0.8 * 0.8)}, val: {int(len(dataset) * 0.8 * 0.2)}, test: {int(len(dataset) * 0.2)}")

total: 55777
train: 35697, val: 8924, test: 11155


36k images, 32bit floats, 256*256*2 numbers: 19 gigabytes total

In [7]:
36e3*4*256*256*2/1e9

18.874368

at batch size 32, 16 workers, rose-bird-52 used about 4.5GB at steady state. Looks like the readout in the chart is gigabits?

In [12]:
16 * 2 * 32 * 256 * 256 * 2 * 4 / 1e9

0.536870912

In [13]:
16 * 2 * 32 * 256 * 256 * 2 * 4 / 1e9 * 8

4.294967296

In [10]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:        1031875       24179      945916         222       61779     1002191
Swap:             0           0           0


So there is like 1T of memory

So for the datasize we have the following number of batches and how many will be covered by the number workers? Does it exeed the system memory?

In [22]:
batch_size = 24
num_workers = 256
prefetch_factor = 2

batch_n = int(len(dataset) * 0.8 * 0.8 / batch_size) + 1
print(f"Number of batches: {batch_n}")
print(f"Workers can concurrently load {num_workers * prefetch_factor} batches")
print(f"Which means {int(batch_n / (num_workers * prefetch_factor)) + 1} loading periods will be needed")
print(f"At one time, they will need {num_workers * prefetch_factor * batch_size * res * res * 2 * 32 / 1e9} GiB of memory")

Number of batches: 1488
Workers can concurrently load 512 batches
Which means 3 loading periods will be needed
At one time, they will need 51.539607552 GiB of memory


Honestly this will probably be limited by the process memory and disk bandwidth to begin with. 256 is too much.
With a bigger batchsize, the model computation time will be longer so you can have less workers. But the upfront time might be too big. Could be better to do tiny batches and a shit ton of workers...